In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model_str import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)
    
N_EPOCHS = 50
N_TRIALS = 30
BATCH_SIZE = 512

CONFIG = config_tests["proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
ds_1 = tfds.load("eurosat", split="train", as_supervised=True, shuffle_files=False)

x = []
y = []
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
    if len(x) == 300:
        break

2024-06-01 14:31:30.012513: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:
x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(210, 45, 45)

In [5]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [6]:
x_train[0].shape

(64, 64, 3)

In [7]:
input_shape=(None, 64, 64, 3), 

In [8]:
model = ClassifierModel(input_shape=(64, 64, 3), output_shape=n_classes, **models_hyperparameters["test_model_1"])

In [9]:
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [10]:
model.layers

[<InputLayer name=image, built=True>,
 <ConvBlock name=conv_block, built=True>,
 <ConvBlock name=conv_block_1, built=True>,
 <ConvBlock name=conv_block_2, built=True>,
 <Flatten name=flatten, built=True>,
 <DenseBlock name=dense_block, built=True>,
 <Dense name=dense_1, built=True>]

In [21]:
bb = 1

for layer in model.layers[1:6]:
    layer.name = f"Block {bb}"
    bb += 1
model.layers[0].name = "Input"
model.layers[4].name = "Flatten"
model.layers[-1].name = "Output"


for layer in model.layers[1:4]:   
    ii = 1
    for ll in layer.layers:
        ll.name = f"Layer {ii}"
        ii += 1
        
ii = 1
for ll in model.layers[5].layers:
    ll.name = f"Layer {ii}"
    ii += 1

model.layers[-1] = f"Layer {ii}"

In [22]:
from tensorflow.python.keras.utils import vis_utils

In [38]:
def m2tex():
    stringlist = []
    model.summary(line_length=85, print_fn=(lambda x: stringlist.append(x))())
    del stringlist[1:-4:2]
    del stringlist[-1]
    for ix in range(1,len(stringlist)-3):
        tmp = stringlist[ix]
        stringlist[ix] = tmp[0:31]+"& "+tmp[31:59]+"& "+tmp[59:]+"\\\\ \hline"
    stringlist[0] = "Model: test \\\\ \hline"
    stringlist[1] = stringlist[1]+" \hline"
    stringlist[-4] = stringlist[-4]+" \hline"
    stringlist[-3] = stringlist[-3]+" \\\\"
    stringlist[-2] = stringlist[-2]+" \\\\"
    stringlist[-1] = stringlist[-1]+" \\\\ \hline"
    prefix = ["\\begin{table}[]", "\\begin{tabular}{lll}"]
    suffix = ["\end{tabular}", "\caption{Model summary for test.}", "\label{tab:model-summary}" , "\end{table}"]
    stringlist = prefix + stringlist + suffix 
    out_str = " \n".join(stringlist)
    out_str = out_str.replace("_", "\_")
    out_str = out_str.replace("#", "\#")
    print(out_str)

m2tex()

TypeError: <lambda>() missing 1 required positional argument: 'x'

In [32]:
def myprint(s):
    with open('abc.txt','w') as f:
        print(s, file=f)
        
model.summary(line_length=85, expand_nested=True, positions=[0.15, 0.25, 0.30, 0.5], print_fn=myprint())

TypeError: myprint() missing 1 required positional argument: 's'

In [30]:
from contextlib import redirect_stdout

with open('modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary(line_length=85, expand_nested=True, positions=[0.15, 0.25, 0.30, 0.5])

Model: "classifier_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                  │ (None, 64, 64, 3)           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│ Block 1 (ConvBlock)                 │ (None, 32, 32, 16)          │           512 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 1 (Conv2D)               │ ?                           │           448 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 2 (BatchNormalization)   │ ?                           │            64 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 3 (ReLU)                 │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 4 (Dropout)              │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│ Block 2 (ConvBlock)                 │ (None, 16, 16, 32)          │         4,768 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 1 (Conv2D)               │ ?                           │         4,640 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 2 (BatchNormalization)   │ ?                           │           128 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 3 (ReLU)                 │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 4 (Dropout)              │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│ Block 3 (ConvBlock)                 │ (None, 8, 8, 64)            │        18,752 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 1 (Conv2D)               │ ?                           │        18,496 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 2 (BatchNormalization)   │ ?                           │           256 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 3 (ReLU)                 │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 4 (Dropout)              │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│ Flatten (Flatten)                   │ (None, 4096)                │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│ Block 5 (DenseBlock)                │ (None, 32)                  │       131,232 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 1 (Dense)                │ ?                           │       131,104 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 2 (BatchNormalization)   │ ?                           │           128 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 3 (ReLU)                 │ ?                           │             0 │
├─────────────────────────────────────┼─────────────────────────────┼───────────────┤
│    └ Layer 4 (Dropout)              │ ?                           │             0 │
├─────────────────────────────────────┼─────

 Total params: 155,594 (607.79 KB)

 Trainable params: 155,306 (606.66 KB)

 Non-trainable params: 288 (1.12 KB)

In [ ]:
Non-trainable params: 288a